In [1]:
# Loading requisite libraries
import selenium
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import os

In [4]:
# List the path to chromedriver
path = "D:/Arivu/Python/github/web-scraper_python/chromedriver"
        
#Initializing the webdriver
options = webdriver.ChromeOptions()
     
#Change the path to where chromedriver is in your home folder.
driver = webdriver.Chrome(executable_path=path, options=options)
driver.set_window_size(1120, 1000)

# Website url which has to be scrapped
# In this example, the objective is to extract data from glassdoor website
url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + "Data-Scientist" + '"&locT=C&locId=1147401&locKeyword=san+fransico,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
driver.get(url)
jobs = []

# Click anywhere on the webpage - Selected to click detailsalary on the left pane
try:
    driver.find_element_by_css_selector('[data-test="detailSalary"]').click()
except ElementClickInterceptedException:
    pass
time.sleep(.1)

# Pop-up window comes towards login; close the window automatically by presssing the cross button
try:
    driver.find_element_by_css_selector('[alt="Close"]').click()
except NoSuchElementException:
    pass
time.sleep(.1)

# Select the entire listing on the left pane
job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")
df = pd.DataFrame()

<ipython-input-4-e20cff20cf14>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)
<ipython-input-4-e20cff20cf14>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('[data-test="detailSalary"]').click()
<ipython-input-4-e20cff20cf14>:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('[alt="Close"]').click()
<ipython-input-4-e20cff20cf14>:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")


In [5]:
def parse_site():
    job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")
    x=1
    jobs.clear()
    
    # Select each of the jobs listed on the left pane
    for job_button in job_buttons: 
        
        # Select and store the requisite data from the webpage
        try:
            company_name = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/div[1]/a').text
            location = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/div[2]/span').text
            job_title = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/a/span').text        
        except:
            time.sleep(5)

        try:
            rating = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[1]/span').text
        except NoSuchElementException:
            rating = -1

        try:
            salary_est = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/div[3]/div[1]/span').text
        except NoSuchElementException:
            salary_est = -1

        job_button.click()
        time.sleep(15)
        
        # Once data is collected on the left pane, the particular job listing is clicked so as to get the detailed listing on the right pane
        try:
            founded = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[2]/span[2]').text
        except NoSuchElementException:
            founded = -1

        try:
            size = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]').text
        except NoSuchElementException:
            size = -1

        try:
            sector = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span[2]').text
        except NoSuchElementException:
            sector = -1

        try:
            revenue = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span[2]').text
        except NoSuchElementException:
            revenue = -1

        try:
            industry = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]').text
        except NoSuchElementException:
            industry = -1

        try:
            type_of_ownership = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span[2]').text
        except NoSuchElementException:
            type_of_ownership = -1 

        # Append to the data-frame
        jobs.append({"Job Title" : job_title,
                     "Company Name" : company_name,
                     "Salary Estimate" : salary_est,
                     "Rating" : rating,
                     "Location" : location,
                     "Founded In": founded,
                     "Company Size": size,
                     "Sector": sector,
                     "Revenue": revenue,
                     "Industry": industry,
                     "Ownership_type": type_of_ownership
                    })
        
        # Print the company name & respective details to show the status
        print(company_name +" I "+location+" I "+job_title)
        print(x)
        x += 1
 
    return pd.DataFrame(jobs)

In [6]:
x=1
while (x<21):
    temp = parse_site()
    df = df.append(temp, ignore_index=True)
    
    # Once the entire jobs listed on the 1st tab is recorded, selecting the 2nd tab for job listing
    try:
        driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a/span').click()
        x += 1
    except:
        x = 21
    time.sleep(15)

df

<ipython-input-5-e61a50cc0204>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  job_buttons = driver.find_elements_by_css_selector("li.react-job-listing")
<ipython-input-5-e61a50cc0204>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  company_name = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/div[1]/a').text
<ipython-input-5-e61a50cc0204>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  location = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/div[2]/span').text
<ipython-input-5-e61a50cc0204>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  job_title = driver.find_element_by_xpath('//*[@id="MainCol"]/div[1]/ul/li['+str(x)+']/div[2]/a/span').text
<ipython-input-5-e61a50cc0204>:14: DeprecationWarning: find_e

Urban Alchemy I San Francisco, CA I SF Practitioner
1
TSA (Transportation Security Administration) I Oakland, CA I Transportation Security Inspector
2
TSA (Transportation Security Administration) I San Francisco, CA I Administrative Assistant
3
Federal Bureau of Investigation I San Francisco, CA I Special Agent
4
California State University I San Francisco, CA I Warehouse Worker
5
TikTok I Mountain View, CA I Product Policy Manager, Adult Nudity & Sexual Activity
6
The Heising-Simons Foundation I San Francisco, CA I Program Associate, Climate & Clean Energy
7
City of Fremont I Fremont, CA I Park Maintenance Worker I
8
visit-healthcare I Daly City, CA I COVID Test Specialists needed ASAP!
9
Excellence Services, LLC I Atherton, CA I Remote Personal Assistant
10
Trader Joe's I San Francisco, CA I Crew
11
City of Fremont I Fremont, CA I Community Development Technician
12
Cash App I San Francisco, CA I Voice of the Customer Specialist
13
Sick.org I San Francisco, CA I Delivery Driver - Cov

<ipython-input-6-00ce20605e76>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a/span').click()


Tinder I San Francisco, CA I Data Scientist I, Product Analytics
1
Slack I San Francisco, CA I Staff Data Scientist, Virtual HQ
2
IBM I San Jose, CA I Junior Data Scientist Apprentice
3
Pinterest I San Francisco, CA I University Grad Data Scientist
4
Gap Inc. I San Francisco, CA I Associate Data Scientist
5
TikTok I Mountain View, CA I Data Scientist
6
FriendFinder Networks I San Jose, CA I Data Scientist
7
Intel I Santa Clara, CA I Data Scientist
8
Grammarly, Inc. I San Francisco, CA I Data Scientist
9
Microsoft I Mountain View, CA I Data Scientist
10
Slack I San Francisco, CA I Senior Data Scientist, Slack Connect
11
Levi Strauss & Co. I San Francisco, CA I Data Scientist I
12
The Patrick J. McGovern Foundation I San Francisco, CA I Data Scientist (Remote)
13
Visa I Foster City, CA I Data Scientist
14
Avila Trading I San Francisco, CA I Data scientist
15
Google I Mountain View, CA I Data Scientist, Operations Data Science
16
Roku I San Jose, CA I Data Scientist, TV
17
Abbott Laborato

WebDriverException: Message: chrome not reachable
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E1FDC3+2555331]
	Ordinal0 [0x00DB77F1+2127857]
	Ordinal0 [0x00CB2CD0+1060048]
	Ordinal0 [0x00CA7BE2+1014754]
	Ordinal0 [0x00CA83E8+1016808]
	Ordinal0 [0x00CA9C02+1022978]
	Ordinal0 [0x00CA3AB9+998073]
	Ordinal0 [0x00CB4040+1065024]
	Ordinal0 [0x00D06DC2+1404354]
	Ordinal0 [0x00CF7976+1341814]
	Ordinal0 [0x00CD36B6+1193654]
	Ordinal0 [0x00CD4546+1197382]
	GetHandleVerifier [0x00FB9622+1619522]
	GetHandleVerifier [0x0106882C+2336844]
	GetHandleVerifier [0x00EB23E1+541697]
	GetHandleVerifier [0x00EB1443+537699]
	Ordinal0 [0x00DBD18E+2150798]
	Ordinal0 [0x00DC1518+2168088]
	Ordinal0 [0x00DC1660+2168416]
	Ordinal0 [0x00DCB330+2208560]
	BaseThreadInitThunk [0x75DFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77347A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77347A6E+238]


In [195]:
# Storing to an excel file
path = os.path.join(os.getcwd(),'glassdoor_job_postings.xls')
df.to_excel(path, index=False)